# Neural network prediction
Source:
https://github.com/kristpapadopoulos/seriesnet.git

In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Conv1D, Input, Add, Activation, Dropout

from keras.models import Sequential, Model

from keras.regularizers import l2

from keras.initializers import TruncatedNormal

from keras.layers.advanced_activations import LeakyReLU, ELU

from keras import optimizers

Using TensorFlow backend.


In [2]:
def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def f(input_):
        
        residual =    input_
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False,
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(input_)
                    
        layer_out =   Activation('selu')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
        
        network_in =  Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
                      
        network_out = Add()([residual, network_in])
        
        return network_out, skip_out
    
    return f


In [3]:
def DC_CNN_Model(length):
    
    input = Input(shape=(length,1))
    
    l1a, l1b = DC_CNN_Block(32,2,1,0.001)(input)    
    l2a, l2b = DC_CNN_Block(32,2,2,0.001)(l1a) 
    l3a, l3b = DC_CNN_Block(32,2,4,0.001)(l2a)
    l4a, l4b = DC_CNN_Block(32,2,8,0.001)(l3a)
    l5a, l5b = DC_CNN_Block(32,2,16,0.001)(l4a)
    l6a, l6b = DC_CNN_Block(32,2,32,0.001)(l5a)
    l6b = Dropout(0.8)(l6b) #dropout used to limit influence of earlier data
    l7a, l7b = DC_CNN_Block(32,2,64,0.001)(l6a)
    l7b = Dropout(0.8)(l7b) #dropout used to limit influence of earlier data

    l8 =   Add()([l1b, l2b, l3b, l4b, l5b, l6b, l7b])
    
    l9 =   Activation('relu')(l8)
           
    l21 =  Conv1D(1,1, activation='linear', use_bias=False, 
           kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=42),
           kernel_regularizer=l2(0.001))(l9)

    model = Model(inputs=input, outputs=l21)
    
    adam = tf.keras.optimizers.Adam(lr=0.00075, beta_1=0.9, beta_2=0.999, epsilon=None,  decay=0.0, amsgrad=False)
    
    #model.compile(loss='kullback_leibler_divergence', optimizer=adam, metrics=['mse']) #not so good
    model.compile(loss='mae', optimizer=adam, metrics=['mse']) # best
    
    return model

In [4]:
#url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df = pd.read_csv(url)
df2 = df.loc[(df["Country/Region"] == "Italy"),'2/29/20':]
#df2.loc[:,'3/23/20']= [167];
#df2 = df2.assign(next = [9134]) 
arr = df2.to_numpy()
#arr = list(np.arange(60,df2.shape[1]+60))
arr = arr[arr != 0]
print(arr.shape[1])
i = 0
b = []
while i < arr.shape[1]:
    temp = arr[0][i]-arr[0][i-1]
    if temp > 0:
        b.append(temp)
    i += 1
timeseries = np.array(b)
timeseries

# timeseries input is 1-D numpy array
# forecast_size is the forecast horizon

timeseries = timeseries[~pd.isna(timeseries)]

length = len(timeseries)-1

timeseries = np.atleast_2d(np.asarray(timeseries))
if timeseries.shape[0] == 1:
    timeseries = timeseries.T 

model = DC_CNN_Model(length)
print('\n\nModel with input size {}, output size {}'.
                            format(model.input_shape, model.output_shape))

model.summary()

X = timeseries[:-1].reshape(1,length,1)
y = timeseries[1:].reshape(1,length,1)

model.fit(X, y, epochs=3000)
   


66.8750
Epoch 2011/3000
1/1 [==============================] - 0s 5ms/step - loss: 57.9946 - mse: 8302.7998
Epoch 2012/3000
1/1 [==============================] - 0s 5ms/step - loss: 57.9247 - mse: 8299.8770
Epoch 2013/3000
1/1 [==============================] - 0s 5ms/step - loss: 57.8866 - mse: 8292.9746
Epoch 2014/3000
1/1 [==============================] - 0s 5ms/step - loss: 57.9953 - mse: 8271.4912
Epoch 2015/3000
1/1 [==============================] - 0s 4ms/step - loss: 58.0350 - mse: 8311.6924
Epoch 2016/3000
1/1 [==============================] - 0s 5ms/step - loss: 57.9382 - mse: 8309.9795
Epoch 2017/3000
1/1 [==============================] - 0s 5ms/step - loss: 57.9231 - mse: 8306.5811
Epoch 2018/3000
1/1 [==============================] - 0s 5ms/step - loss: 58.0906 - mse: 8247.9844
Epoch 2019/3000
1/1 [==============================] - 0s 4ms/step - loss: 58.1430 - mse: 8225.8896
Epoch 2020/3000
1/1 [==============================] - 0s 3ms/step - loss: 58.1417 - mse: 82

In [5]:
predict_size = 30

pred_array = np.zeros(predict_size).reshape(1,predict_size,1)
X_test_initial = timeseries[1:].reshape(1,length,1)
print(X_test_initial)
pred_array = model.predict(X_test_initial)# if predictions of training samples required

#forecast is created by predicting next future value based on previous predictions
pred_array[:,0,:] = model.predict(X_test_initial)[:,-1:,:]
for i in range(predict_size-1):
    print(i)
    print(pred_array)
    pred_array[:,i+1:,:] = model.predict(np.append(X_test_initial[:,i+1:,:], 
                           pred_array[:,:i+1,:]).reshape(1,length,1))[:,-1:,:]


[[[ 18]
  [ 27]
  [ 28]
  [ 41]
  [ 49]
  [ 36]
  [133]
  [ 97]
  [168]
  [196]
  [439]
  [175]
  [368]
  [349]
  [345]
  [475]
  [427]
  [627]
  [793]
  [651]
  [601]
  [743]
  [683]
  [712]
  [919]
  [889]
  [756]
  [812]
  [837]
  [727]]]
0
[[[752.4403  ]
  [ 37.068474]
  [ 46.37473 ]
  [ 62.192192]
  [ 76.27812 ]
  [ 80.60311 ]
  [ 96.95216 ]
  [123.6848  ]
  [155.91197 ]
  [195.40271 ]
  [265.8836  ]
  [292.66867 ]
  [349.89038 ]
  [346.50452 ]
  [405.406   ]
  [454.691   ]
  [474.36557 ]
  [545.72845 ]
  [651.19165 ]
  [703.9364  ]
  [747.4763  ]
  [744.06195 ]
  [723.3571  ]
  [782.6226  ]
  [809.29205 ]
  [839.7951  ]
  [807.33594 ]
  [841.13794 ]
  [728.9237  ]
  [752.4403  ]]]
1
[[[752.4403]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [718.6165]
  [

In [6]:
print(X)
print(y)

[[[  5]
  [ 18]
  [ 27]
  [ 28]
  [ 41]
  [ 49]
  [ 36]
  [133]
  [ 97]
  [168]
  [196]
  [439]
  [175]
  [368]
  [349]
  [345]
  [475]
  [427]
  [627]
  [793]
  [651]
  [601]
  [743]
  [683]
  [712]
  [919]
  [889]
  [756]
  [812]
  [837]]]
[[[ 18]
  [ 27]
  [ 28]
  [ 41]
  [ 49]
  [ 36]
  [133]
  [ 97]
  [168]
  [196]
  [439]
  [175]
  [368]
  [349]
  [345]
  [475]
  [427]
  [627]
  [793]
  [651]
  [601]
  [743]
  [683]
  [712]
  [919]
  [889]
  [756]
  [812]
  [837]
  [727]]]


In [7]:
pred_array

array([[[752.4403 ],
        [718.6165 ],
        [689.9553 ],
        [682.205  ],
        [662.87476],
        [652.8758 ],
        [638.1908 ],
        [624.30396],
        [611.4985 ],
        [600.89136],
        [586.8496 ],
        [575.29694],
        [564.2364 ],
        [552.6883 ],
        [542.1879 ],
        [532.03894],
        [522.4168 ],
        [512.87006],
        [504.03677],
        [495.37338],
        [487.1014 ],
        [479.13745],
        [471.45316],
        [464.11438],
        [457.01627],
        [450.21274],
        [443.67154],
        [437.39   ],
        [431.34186],
        [425.54047]]], dtype=float32)

In [8]:
pred_array.ravel()

array([752.4403 , 718.6165 , 689.9553 , 682.205  , 662.87476, 652.8758 ,
       638.1908 , 624.30396, 611.4985 , 600.89136, 586.8496 , 575.29694,
       564.2364 , 552.6883 , 542.1879 , 532.03894, 522.4168 , 512.87006,
       504.03677, 495.37338, 487.1014 , 479.13745, 471.45316, 464.11438,
       457.01627, 450.21274, 443.67154, 437.39   , 431.34186, 425.54047],
      dtype=float32)

## Some Results
``` array([[[ 9831.691  ],
        [10554.103  ],
        [11080.334  ],
        [11511.992  ],
        [11715.157  ],
        ```

In [9]:
summe = np.array([828.422  , 712.6765 , 708.49805, 675.9196 , 615.71423, 530.84576,
       490.84058, 473.8505 , 422.53046, 402.77542, 378.3785 , 347.97464,
       331.39874, 312.33398, 302.99292, 288.85974, 278.36542, 272.8782 ,
       262.7774 , 261.38535, 254.58046, 251.12053, 248.2424 , 243.78625,
       243.97864, 240.55757, 242.22415, 243.51944, 240.1935 , 239.41483])

summe.sum()
len(summe)
len(pred_array)

1